In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, GlueDataset, default_data_collator
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import mlflow
import matplotlib.pyplot as plt

/Applications/anaconda3/envs/BERT-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("../data/train_df.csv")
data = data.dropna(subset=['words'])
data = data.sample(10000)

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len = 128

/Applications/anaconda3/envs/BERT-test/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def encode_text(texts, tokenizer, max_len):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_len,
                            padding='max_length',
                            return_attention_mask=True,
                            return_tensors='pt',
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

In [5]:
input_ids, attention_masks = encode_text(data['words'], tokenizer, max_len)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, data['target'].values, test_size=0.1, random_state=42)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.1, random_state=42)

In [6]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
eval_dataset = TensorDataset(validation_inputs, validation_masks, validation_labels)

/var/folders/kl/3nc79ycx61v13jbqlw61nnfw0000gn/T/ipykernel_33031/2933568192.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs)
/var/folders/kl/3nc79ycx61v13jbqlw61nnfw0000gn/T/ipykernel_33031/2933568192.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_inputs = torch.tensor(validation_inputs)
/var/folders/kl/3nc79ycx61v13jbqlw61nnfw0000gn/T/ipykernel_33031/2933568192.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_masks = torch.tensor(train_masks)
/var/folders/kl/3nc79ycx61

In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def my_data_collator(features):
    batch = {}
    batch['input_ids'] = torch.stack([f[0] for f in features])
    batch['attention_mask'] = torch.stack([f[1] for f in features])
    batch['labels'] = torch.tensor([f[2] for f in features])
    return batch

In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    warmup_steps=0,
    weight_decay=0.01,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

In [10]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_predictions):
    labels = eval_predictions.label_ids
    preds = eval_predictions.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {
        'eval_accuracy': accuracy,
    }

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=my_data_collator,
    compute_metrics=compute_metrics
)

In [12]:
trainer.train()

  1%|          | 10/846 [02:23<3:14:27, 13.96s/it]

{'loss': 0.7017, 'grad_norm': 5.859342098236084, 'learning_rate': 1.9763593380614657e-05, 'epoch': 0.04}


  2%|▏         | 20/846 [04:49<3:13:45, 14.07s/it]

{'loss': 0.6969, 'grad_norm': 2.4730896949768066, 'learning_rate': 1.9527186761229316e-05, 'epoch': 0.07}


  4%|▎         | 30/846 [07:13<3:10:55, 14.04s/it]

...

 99%|█████████▉| 840/846 [3:40:18<01:25, 14.19s/it]

{'loss': 0.3891, 'grad_norm': 4.673222064971924, 'learning_rate': 1.4184397163120568e-07, 'epoch': 2.98}


100%|██████████| 846/846 [3:41:35<00:00, 15.72s/it]

{'train_runtime': 13296.0218, 'train_samples_per_second': 2.031, 'train_steps_per_second': 0.064, 'train_loss': 0.4437316661749044, 'epoch': 3.0}


TrainOutput(global_step=846, training_loss=0.4437316661749044, metrics={'train_runtime': 13296.0218, 'train_samples_per_second': 2.031, 'train_steps_per_second': 0.064, 'total_flos': 1775999623680000.0, 'train_loss': 0.4437316661749044, 'epoch': 3.0})

In [13]:
model_path = "./model"
tokenizer_path = "./tokenizer"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.txt',
 './tokenizer/added_tokens.json')

In [14]:
tokenizer_path = "./tokenizer"
model_path = "./model"

tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
model = BertForSequenceClassification.from_pretrained(model_path)

In [15]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
text = "Hello, how are you?"

result = classifier(text)
print(result)

[{'label': 'LABEL_1', 'score': 0.8972613215446472}]


In [16]:
text = "I want you to die"

result = classifier(text)
print(result)

[{'label': 'LABEL_0', 'score': 0.7834637761116028}]


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import torch
from transformers import BertTokenizer, BertForSequenceClassification

data = pd.read_csv("../Data/train_df.csv")
data = data.dropna(subset=['words'])
data = data.sample(1000)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("chemin/vers/votre/modele")

# Prétraiter les données de test
test_texts = ["Hello, how are you?"]  # Mettez vos propres données de test ici
test_input_ids = []
test_attention_masks = []
for text in test_texts:
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens=True,
                        max_length=128,
                        padding='max_length',
                        return_attention_mask=True,
                        return_tensors='pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

# Effectuer les prédictions
with torch.no_grad():
    outputs = model(test_input_ids, attention_mask=test_attention_masks)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# Calculer les métriques de performance
accuracy = accuracy_score(data['target'], predictions)
precision = precision_score(data['target'], predictions)
recall = recall_score(data['target'], predictions)
f1 = f1_score(data['target'], predictions)
fpr, tpr, thresholds = roc_curve(data['target'], predictions)
roc_auc = auc(fpr, tpr)
conf_matrix = confusion_matrix(data['target'], predictions)

# Enregistrer les résultats dans MLflow
import mlflow

artifact_path = './artifacts/'

with mlflow.start_run(run_name="bert-base-uncased"):
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1_Score", f1)
    mlflow.log_metric("AUC", roc_auc)

    conf_matrix_path = f"{artifact_path}confusion_matrix.csv"
    pd.DataFrame(conf_matrix).to_csv(conf_matrix_path, index=False, header=False)
    mlflow.log_artifact(conf_matrix_path, "metrics")

    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.grid(True)
    roc_curve_path = f"{artifact_path}roc_curve.png"
    plt.savefig(roc_curve_path)
    plt.close()
    mlflow.log_artifact(roc_curve_path, "plots")
